In [1]:
from tools import *
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from scipy.stats import multivariate_normal
import pandas as pd
import seaborn as sns
from scipy.linalg import fractional_matrix_power
from scipy.stats import wishart
import sys
import time
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
def logpdfTarget (x, mu, covariance_inverse):
    lnp = -(1/2) * ((x-mu).T).dot(covariance_inverse).dot(x-mu)
    return(lnp)

In [3]:
np.random.seed(3)
d = 100
mu_y = true_mu = np.ones(d)
sigma_inverse_y = true_sigma_inverse = wishart.rvs(df=d,scale=np.eye(d))
true_sigma = np.linalg.inv(true_sigma_inverse)
mu_hat = np.zeros(d); Sigma_neg = Sigma_pos = sigma_hat = np.eye(d)
print(np.sum(true_sigma>0.99)/2)
true_sigma

1596.5


array([[ 8.48568807, -7.95449213,  6.48567873, ...,  5.51870081,
         3.32191262,  2.93964169],
       [-7.95449213,  8.11663935, -6.54354187, ..., -5.35046354,
        -3.4312295 , -2.78842253],
       [ 6.48567873, -6.54354187,  5.66062924, ...,  4.406142  ,
         2.98913136,  2.19140341],
       ...,
       [ 5.51870081, -5.35046354,  4.406142  , ...,  3.90847541,
         2.39666249,  1.86435636],
       [ 3.32191262, -3.4312295 ,  2.98913136, ...,  2.39666249,
         1.8768823 ,  1.11750191],
       [ 2.93964169, -2.78842253,  2.19140341, ...,  1.86435636,
         1.11750191,  1.43581233]])

In [4]:
np.random.seed(121)
begin_total = time.time()
nround = int(10e5); round_range = np.arange(nround)
burnin = int(10e5); estimate_round = int(10e4); burnin_range = np.flip(np.arange(-burnin,0))
size = len(range(-burnin,nround))*d

sigmaset = [0.5]
# kernelset = ['Uniform','MirrorU']
kernelset = ['MirrorU']
target_reuslt = {}

eff_kernel = {}; Pjump_kernel = {}; Exx2_kernel = {}; Rho_kernel={}; Sample_kernel={}; Quantile_kernel={}
for kernel in kernelset:
    Eff = []; Pjump = []; Exx2=[]; Rho=[]; Sample=[]
    
    if (kernel == 'Uniform') | (kernel == 'MirrorU'):
        random_result = np.random.uniform(low=-np.sqrt(3), high=np.sqrt(3), size=size)
    elif (kernel == 'Gaussian') | (kernel == 'MirrorN'):
        random_result = np.random.normal(size = size)
    elif kernel == 'Bactrian':
        random_result = random_bactrian(size = size)
    elif kernel == 'StrawHat':
        random_result = random_strawhat(size = size)
    elif kernel == 'Box':
        random_result = random_box(size = size)
    elif kernel == 'Airplane':
        random_result = random_airplane(size = size)
    else:
        sys.exit('No such kernel exists')
    
    for sigma in sigmaset:
#         result_y = np.zeros(nround*d).reshape(nround,d); burn_in_y = np.zeros(burnin*d).reshape(burnin,d)
        begin_loop = time.time(); burn_in_result = np.zeros(burnin*d).reshape(burnin,d); accept = np.zeros(d)
        y = np.random.random(d) - 0.5; result = np.zeros(nround*d).reshape(nround,d)
        logpdf_old = logpdfTarget(y,true_mu,true_sigma_inverse)
        estimate_number = 0
        for j in np.concatenate([burnin_range,round_range]):
            if (j==0):
                count = 0; accept = np.zeros(d);
            
            for i in range(d):
                initial = y.copy()
                if estimate_number == 0:
                    initial[i] += random_result[(j+burnin)*d+i] * sigma
                else:
                    initial[i] = ((2 * mu_star[i] - initial[i]) if 'Mirror' in kernel else initial[i]) + random_result[(j+burnin)*d+i] * sigma
                logpdf_new = logpdfTarget(initial,mu_y,sigma_inverse_y)
                pi_dif = logpdf_new - logpdf_old
                if ((pi_dif>0) | (np.exp(pi_dif)>np.random.random())):
                    y = initial
                    logpdf_old = logpdf_new
                    accept[i] += 1
            xnew = np.dot(Sigma_pos,y)+mu_hat
            if (j>=0):
                result[j,:] = xnew
#                 result_y[j,:] = y
            if (j<0):
                burn_in_result[(-j-1),:] = xnew
#                 burn_in_y[(-j-1),:] = y
                
            if ((j % estimate_round == 0) & (j<0)):
                estimate_number += 1
                mu_hat_old = mu_hat; Sigma_pos_old = Sigma_pos;
                mu_hat = np.mean(burn_in_result[:(-j),:],0);
                sigma_hat = np.cov(burn_in_result[:(-j),:].T)
                Sigma_neg = fractional_matrix_power(sigma_hat,-0.5); Sigma_pos = fractional_matrix_power(sigma_hat,0.5)
                mu_star = mu_y = Sigma_neg.dot(true_mu-mu_hat)
                sigma_inverse_y = np.linalg.inv(Sigma_neg.dot(true_sigma).dot(Sigma_neg.T))
                y = Sigma_neg.dot(Sigma_pos_old.dot(y) + mu_hat_old - mu_hat)
                print(mu_hat,mu_star)
                print(sigma_hat)
                print('-------------')

        eff = Eff_IntegratedCorrelationTime(result[:,0])
        exx2 = sum(np.square(result[:,0][1:] - result[:,0][:-1]))
        rho = [rho_k(result[:,0],i) for i in range(1,11)]
        Eff.append(eff)
        Pjump.append(accept[0]/nround)
        Exx2.append(exx2/nround)
        Rho.append(rho)
        Sample.append(result[:,0].copy())
        print ('Kernel:',kernel,'|sigma:',sigma,'|eff:',round(eff,4), '|Pjump:', round(accept[0]/nround,4), '|Exx2:', round(exx2/nround,4),
               '|rho1:', round(rho[0],4), '|rho2:', round(rho[1],4),'|Time:',round((time.time()-begin_loop),2),sep='')
    #     print('协方差矩阵: \n',np.cov(np.vstack([y[:,i] for i in range(d)])))
    eff_kernel[kernel] = Eff
    Pjump_kernel[kernel] = Pjump
    Exx2_kernel[kernel] = Exx2
    Rho_kernel[kernel] = Rho
    Sample_kernel[kernel] = Sample
print('总共用时: %f'% (time.time()-begin_total))

[ 2.76977055 -1.09356902  2.75057807 -0.48611345  1.87282001  1.09299115
 -1.48910599 -0.96961813 -0.1671824  -1.4856852   2.88079502  1.72338193
  1.23137745  1.67966608  1.44154403 -0.76341751  2.47473757  2.21262519
  2.14143106  2.49094112  1.21677171  1.07303371  1.4697578   0.88210827
  0.32924357  1.31694568  1.17563924  2.80566241  0.91109054  3.89617281
 -0.51620902  1.50154743 -0.89837303 -0.11945152  1.33564426  0.21247723
  0.4539659   1.7666396   1.90317489  3.09850874  3.14463472 -1.4042399
  2.80842358 -0.85066721  2.38929409  1.53583811  2.80433207  1.78729959
  1.17034829  0.55052036 -1.80741587  3.34163226  2.49307832  0.38102769
 -0.31626804 -0.96652914  2.3617801   2.85080436  1.08011251  1.54703002
  1.29400917  1.65736682  1.60885163 -1.04618396  3.57177746  0.7146281
 -2.03407044  1.62623741  2.30285937 -1.42621965  4.02707913  1.07999311
  1.39754481  2.3039631   1.7379846   1.62965934  1.12848411  0.15475929
  0.85293023  2.34112703  0.07951538 -0.26663217  0.7

[1.45495327 0.4639003  1.44732349 0.61779772 1.22218746 1.02368976
 0.36257013 0.49815311 0.70146489 0.36608271 1.47981149 1.18682664
 1.06215538 1.17420307 1.11305155 0.54802429 1.37647548 1.30836595
 1.29111912 1.379432   1.05504098 1.01862952 1.12241687 0.96868576
 0.82712797 1.08233364 1.04419621 1.46613285 0.97764295 1.74159905
 0.61219421 1.12737783 0.51464442 0.71235019 1.08472507 0.79861961
 0.86072711 1.19879989 1.23068744 1.53682864 1.54706326 0.3865985
 1.46274898 0.52498707 1.35643458 1.13705449 1.4661888  1.20236545
 1.04396244 0.88408433 0.28152893 1.59980666 1.38584245 0.84491804
 0.66294626 0.49903082 1.34977246 1.47517154 1.02309649 1.1410816
 1.07450541 1.16763174 1.15728106 0.47480877 1.65966245 0.92591895
 0.22360585 1.15946335 1.33325366 0.37739033 1.77474631 1.01794513
 1.10137027 1.33536982 1.1881568  1.16237084 1.03311475 0.78397208
 0.96242466 1.34293036 0.76550536 0.67597136 0.94185562 1.11969627
 0.5458247  0.37875772 0.5455963  0.63806774 0.10155278 1.033678

[1.2596467  0.69405078 1.25560203 0.78204209 1.12735509 1.01351663
 0.63603182 0.71342856 0.82973838 0.63779649 1.27407593 1.10699005
 1.03491624 1.09944512 1.0648222  0.74177104 1.2148155  1.17651708
 1.16622696 1.21683872 1.03141993 1.01060883 1.0696862  0.98201416
 0.90124443 1.04685249 1.02530493 1.2655782  0.98729122 1.42322039
 0.77838207 1.07291247 0.72266711 0.83573184 1.04832263 0.88505762
 0.92037411 1.11315746 1.13158429 1.30701288 1.31245431 0.64952639
 1.2645495  0.72899412 1.20312706 1.07827467 1.26539008 1.11531455
 1.02501206 0.93366697 0.58954242 1.34237462 1.21971331 0.911327
 0.80790244 0.71407382 1.19970405 1.27134765 1.01246817 1.08082698
 1.04216759 1.09586652 1.08949447 0.70018462 1.37659324 0.957777
 0.5566417  1.09145489 1.19017924 0.6444798  1.4421935  1.01047398
 1.05752504 1.19139901 1.10767981 1.0924811  1.01890938 0.87683366
 0.9786312  1.1954523  0.86618959 0.81501996 0.96706332 1.0680409
 0.74050882 0.64507084 0.7404149  0.79343803 0.487068   1.01925474


[1.18294523 0.78467832 1.17995909 0.84606029 1.08947984 1.0097402
 0.74402999 0.79827648 0.88016958 0.74535479 1.19236676 1.07581742
 1.02495935 1.06997945 1.04573563 0.81835067 1.15094979 1.12426889
 1.11666226 1.1524446  1.02197257 1.00736776 1.04925013 0.98722002
 0.93048725 1.03306486 1.01781359 1.18750975 0.99126492 1.29789742
 0.84407219 1.05102033 0.80491102 0.884574   1.03381806 0.91921778
 0.94374329 1.07998699 1.09257195 1.21626369 1.21955778 0.75352267
 1.18607127 0.8092858  1.14297063 1.05541637 1.18731768 1.08111121
 1.01780882 0.95340575 0.71134416 1.24082465 1.15490368 0.93811796
 0.86479189 0.79887836 1.14060573 1.19114377 1.00872782 1.05711356
 1.0294072  1.06755264 1.06284096 0.78900329 1.26519587 0.97022012
 0.68779715 1.0640598  1.13356421 0.74963446 1.31115922 1.00705559
 1.04042306 1.13486559 1.07592761 1.06506441 1.01368066 0.91352717
 0.98505674 1.13751769 0.90604997 0.86990057 0.97676705 1.04826997
 0.81740081 0.75004066 0.81748803 0.8544965  0.63916944 1.01405

In [6]:
print(np.mean(result,0))
print('---------------------------------------------------------')
print(np.cov(result.T))

[0.99841855 1.00178168 0.99884745 1.0003503  0.99935591 1.0004711
 1.00244293 1.0011301  1.00125854 1.00186176 0.99851109 1.00000623
 0.99980601 0.99927554 0.99994157 1.00099272 0.99902924 0.99915361
 0.99886361 0.99941424 0.99966205 0.99965603 1.00011812 0.9996065
 1.00039043 0.99917328 0.99975544 0.99920951 0.99981234 0.99807003
 1.00156242 0.99927362 1.00129215 1.00130903 0.99948658 1.00046628
 1.00031488 0.99938781 0.99902104 0.99923035 0.9981735  1.00184535
 0.99890345 1.00123321 0.9985802  0.9995511  0.99870814 0.99956779
 0.99996058 1.00107541 1.00218649 0.99850932 0.99898443 0.99973263
 1.00124151 1.00067577 0.9990768  0.99874018 0.99960981 0.99953874
 0.99928732 0.99970986 0.99940649 1.001605   0.99781269 0.99974213
 1.00196156 0.99921366 0.99900339 1.00218181 0.99799119 0.99973537
 0.9997038  0.99908073 0.99998865 0.99970004 0.9995982  1.00059127
 0.9995704  0.99942958 1.00090131 1.00082958 1.00023761 0.99976813
 1.00132034 1.00179045 1.00102972 1.00119824 1.00278334 1.000370